In [1]:
from bs4 import BeautifulSoup as bs
from functools import reduce
from pandas.io.json import json_normalize
import json
import matplotlib as mpl
import pandas as pd
import re
import requests
import seaborn as sns
import unicodedata as uni
import os

In [2]:
def read_json_to_dict(filename, path_to_file = None, encoding = 'utf-8'):
    
    path_to_file = reduce(lambda x, y: x if x is not None else y, [path_to_file, os.getcwd().replace('\\', '/')])
    
    if path_to_file[-1:] != '/':
        path_to_file = path_to_file + '/'
        
    full_file_path = path_to_file + '/' + filename
    
    with open(full_file_path, 'r', encoding = encoding) as file:
        data = json.load(file)
    
    return data
        

In [3]:
linneburg_ratings = read_json_to_dict('linneburg.json', 'movie_jsons')

In [4]:
linneburg_ratings

{'account': 'joshlinneburg',
 'account_url': 'https://letterboxd.com/joshlinneburg',
 'account_ratings_url': 'https://letterboxd.com/joshlinneburg/films/ratings/by/rating/',
 'movies_rated': [{'title': 'Chernobyl',
   'url': 'https://letterboxd.com//film/chernobyl/',
   'genres': ['drama'],
   'description': 'The true story of one of the worst man-made catastrophes in history: the catastrophic nuclear accident at Chernobyl. A tale of the brave men and women who sacrificed to save Europe from unimaginable disaster.',
   'avg_rating': '4.5',
   'rating_counts': [{'rating': 0.5,
     'number_of_ratings': 64,
     'perc_of_total': 0.0011},
    {'rating': 1.0, 'number_of_ratings': 66, 'perc_of_total': 0.0012},
    {'rating': 1.5, 'number_of_ratings': 42, 'perc_of_total': 0.0007},
    {'rating': 2.0, 'number_of_ratings': 215, 'perc_of_total': 0.0038},
    {'rating': 2.5, 'number_of_ratings': 260, 'perc_of_total': 0.0046},
    {'rating': 3.0, 'number_of_ratings': 1231, 'perc_of_total': 0.0218

In [5]:
def flatten_crew_dict(movie_dict):

    crew_list = movie_dict['crew']['crew_list']
    crew_wanted_dict = {'director':2, 'producer':3, 'writer':1, 'composer':1}
    flat_crew_dict = {}

    for role, nbr in crew_wanted_dict.items():
        inner_list = []
        try:
            filtered_crew_dict = list(filter(lambda input_list: input_list['crew_role'] == role, crew_list))[0]
        except:
            print('No ' + role + ' found')

        for i in range(nbr):

            try:
                flat_crew_dict[role + '_' + str(i)] = filtered_crew_dict['crew_attributes'][i]['name']
            except IndexError:
                flat_crew_dict[role + '_' + str(i)] = None

    flat_crew_dict['total_crew_size'] = linneburg_ratings['movies_rated'][32]['crew']['total_crew']
    flat_crew_dict['unique_crew_size'] = linneburg_ratings['movies_rated'][32]['crew']['total_unique_crew']

    return flat_crew_dict

In [6]:
def flatten_cast_dict(movie_dict):

    cast_list = movie_dict['cast']['cast_list']
    flat_cast_dict = {}
    cast_to_retrieve = 5

    i = 0
    while i < cast_to_retrieve:
        try:
            flat_cast_dict['actor_'+str(i)] = cast_list[i]['actor_name']
            i += 1
        except:
            flat_cast_dict['actor_'+str(i)] = None


    flat_cast_dict['total_cast_size'] = linneburg_ratings['movies_rated'][32]['cast']['total_cast']
    return flat_cast_dict

In [7]:
def flatten_genre_list(movie_dict):

    genres_list = movie_dict['genres']
    flat_genres_dict = {}
    for i in range(3):
        try:
            flat_genres_dict['genre_'+str(i)] = genres_list[i]
        except IndexError:
            flat_genres_dict['genre_'+str(i)] = None

    return flat_genres_dict

In [8]:
def flatten_ratings_list(movie_dict):

    ratings_list = movie_dict['rating_counts']
    flat_ratings_dict = {}


    for i in range(len(ratings_list)):
        flat_ratings_dict[str(ratings_list[i]['rating']) + '_stars'] = ratings_list[i]['number_of_ratings']

    flat_ratings_dict['total'] = sum(list(flat_ratings_dict.values()))
    return flat_ratings_dict

In [9]:
movies_to_add = linneburg_ratings['movies_rated']
movie_list = []

for i in range(len(movies_to_add)):
    movie_dict = {}
    movie_dict['title'] = movies_to_add[i]['title']
    movie_dict.update(flatten_genre_list(movies_to_add[i]))
    movie_dict['avg_rating'] = movies_to_add[i]['avg_rating']
    movie_dict['user_rating'] = movies_to_add[i]['user_rating']
    movie_dict.update(flatten_crew_dict(movies_to_add[i]))
    movie_dict.update(flatten_cast_dict(movies_to_add[i])) 

    movie_list.append(movie_dict)

No composer found
No composer found


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "<ipython-input-6-12606e06956e>", line 10, in flatten_cast_dict
    flat_cast_dict['actor_'+str(i)] = cast_list[i]['actor_name']
IndexError: list index out of range

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Josh\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3267, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-9-07cdad81face>", line 11, in <module>
    movie_dict.update(flatten_cast_dict(movies_to_add[i]))
  File "<ipython-input-6-12606e06956e>", line 13, in flatten_cast_dict
    flat_cast_dict['actor_'+str(i)] = None
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Josh\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 2018, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'K

TypeError: can only concatenate str (not "list") to str

In [10]:
len(movie_list)

52

In [ ]:
df = json_normalize(movie_list).reindex(columns=list(movie_dict.keys()))
df